In [1]:
import os
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib

In [2]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Arial']
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
sns.set_theme(style="ticks", rc=custom_params)

# matplotlib.rcParams['figure.figsize'] = [5, 5]
# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.verbosity = 3
sc.logging.print_header()
sc.set_figure_params(dpi=300)
%matplotlib inline

2024-10-15 16:22:39.895580: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


scanpy==1.9.6 anndata==0.8.0 umap==0.5.5 numpy==1.23.5 scipy==1.11.4 pandas==1.5.3 scikit-learn==1.4.0 statsmodels==0.14.0 igraph==0.10.3 pynndescent==0.5.8


In [2]:
wd = '/mnt/data/hong/2022/DHJ1_human_obesity_placenta/'

In [3]:
os.chdir(wd)

In [4]:
ad_clean = sc.read_h5ad('output/10x_h5/h5ad/ad_v2.h5ad')

In [86]:
def function_sets(cell_type,function_set, function_name, mode):
    ## read in the data
    # ad_zscore = sc.pp.scale(ad_clean[ad_clean.obs.final_celltypes==cell_type], max_value=10, zero_center=True, layer = 'log_norm', copy = True)
    ad = ad_clean[ad_clean.obs.final_celltypes==cell_type]
    df = pd.read_csv(f'output/DEGs/final_negbinom_all/score_tsv/{cell_type}.tsv', sep='\t')
    ## order df by score
    df.sort_values('score' , ascending=False, inplace=True)
    common_set = df.query("convergence=='high'&p<0.05")['Unnamed: 0']
    divergent_set = df.query("convergence=='low'&p<0.05")['Unnamed: 0']
    common_function = [element for element in common_set if element in function_set]
    divergent_function = [element for element in divergent_set if element in function_set]
    print(len(common_function))
    print(len(divergent_function))
    if mode == 'common':
        function2plot = common_function
    if mode == 'divergent':
        function2plot = divergent_function
    elif mode=='both':
        function2plot = common_function+divergent_function
    if len(function2plot)>0:
        print(function2plot)
        function_name = function_name.replace('/', '_')
        sc.pl.dotplot(ad, function2plot, groupby='group', dendrogram=False, layer = 'log_norm', show=False, swap_axes=True, cmap='YlOrRd', standard_scale='var', save=f'fig2_{cell_type}_{function_name}_standard.pdf', figsize=(2.5, len(function2plot)/2), use_raw=False, categories_order=['Normal_AGA', 'Obese_AGA', 'Obese_LGA'], dot_max
=1, dot_min=0.1)

    # sc.pl.stacked_violin(ad_clean[ad_clean.obs.final_celltypes==cell_type], list(function2plot), groupby='group', dendrogram=False, standard_scale='var', show=False, swap_axes=True, cmap='YlOrRd',save=f'fig2_{cell_type}_{function_name}_violin.pdf', figsize=(2.5, len(function2plot)/2), use_raw=False, categories_order=['Normal_AGA', 'Obese_AGA', 'Obese_LGA'])

## get gene sets from files
The files were downloaded from GO or Hallmark official site. With version registered

In [76]:
def get_set_genes(set_df, set_name):
    gene_str= set_df.loc[set_df.set_name==set_name, 'genes'].to_list()
    gene_list = gene_str[0].split(',')
    return gene_list


In [24]:
genesets = pd.read_csv('data/Placenta_genesets.tsv', sep='\t')
transporters = set(genesets.query("geneset!='secretome'")['genesymbol'])
secretome = set(genesets.query("geneset=='secretome'")['genesymbol'])
tnfa = set(pd.read_csv('data/TNFa.tsv', sep='\t')['SYMBOL'])
hypoxia = set(pd.read_csv('data/hypoxia.tsv', sep='\t')['SYMBOL'])
apical = set(pd.read_csv('data/HALLMARK_APICAL_JUNCTION.v2024.1.Hs.tsv', sep='\t').iloc[16, 1].split(','))
myc = set(pd.read_csv('data/HALLMARK_MYC_TARGETS_V2.v2024.1.Hs.tsv', sep='\t').iloc[16, 1].split(','))
mitotic = set(pd.read_csv('data/HALLMARK_MITOTIC_SPINDLE.v2024.1.Hs.tsv', sep='\t').iloc[16, 1].split(','))
g2m = set(pd.read_csv('data/HALLMARK_G2M_CHECKPOINT.v2024.1.Hs.tsv', sep='\t').iloc[16, 1].split(','))
adipo = set(pd.read_csv('data/HALLMARK_ADIPOGENESIS.v2024.1.Hs.tsv', sep='\t').iloc[16, 1].split(','))
il2 = set(pd.read_csv('data/HALLMARK_IL2_STAT5_SIGNALING.v2024.1.Hs.tsv', sep='\t').iloc[16, 1].split(','))
heme = set(pd.read_csv('data/HALLMARK_HEME_METABOLISM.v2024.1.Hs.tsv', sep='\t').iloc[16, 1].split(','))
hallmark = pd.read_csv('data/MSigDB_Hallmark_2020.tsv', sep='\t', names=['set_name', 'genes'])
kegg = pd.read_csv('data/KEGG_2021_Human.tsv', sep='\t', names=['set_name', 'genes'])
go = pd.read_csv('data/GO_Biological_Process_2023.tsv', sep='\t', names=['set_name', 'genes'])

In [ ]:
set_name = "PI3K-Akt signaling pathway"
function_set = get_set_genes(kegg, set_name)
function_sets('CTB',function_set, set_name, 'common')
set_name = "Positive Regulation Of MAPK Cascade (GO:0043410)"
function_set = get_set_genes(go, set_name) + ['MAPK4']
function_sets('CTB',function_set, set_name, 'divergent')

In [ ]:

function_sets('CTB', function_set, 'Apical Junction', 'common')
function_sets('CTB', function_set,'Mitotic Spindle', 'common')
function_sets('CTB', function_set,'G2-M Checkpoint', 'common')
function_sets('CTB',function_set, 'Adipogenesis', 'common')
function_sets('CTB', function_set,'IL-2/STAT5 Signaling', 'common')
function_sets('CTB',function_set, 'heme Metabolism', 'common')
function_sets('CTB', function_set,'Hypoxia', 'common')
function_sets('CTB',function_set, 'Myc Targets V2', 'divergent')
function_sets('STB-b', transporters, 'transporters', mode='both')
function_sets('STB-c', transporters, 'transporters', mode='both')
function_sets('STB-b', tnfa, 'tnfa', mode='common')
function_sets('STB-c', tnfa, 'tnfa', mode='common')
function_sets('STB-b', hypoxia, 'hypoxia', mode='common')
function_sets('STB-c', hypoxia, 'hypoxia', mode='common')
function_sets('STB-b', secretome, 'secretome', mode='both')
function_sets('STB-c', secretome, 'secretome', mode='both')